In [3]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
data_folder = os.getcwd()+'\\IO'

The file_gams_py_gdb1.gdx is still active and was not deleted.


# *Condition trees on gpy\_symbols*

*The current tutorial outlines a method to slice symbols by setting up condition trees. This slicing using condition trees can be used to either return the sliced variable as a pandas series/index, but can similarly be written to gams code. Thus, condition trees are an efficient way to work on the database from Python, but can similarly be transferred directly to the gams code.*

### 0: Initialize and ready the database/symbols

*Set up database to work on (from excel):*

In [18]:
dsheets = {'Production_v': data_folder+'\\IO_v.xlsx', 'Production_p': data_folder+'\\IO_p.xlsx'}
db = ReadData.read_data.main(dsheets,name='GE_data',components=['domstic','trade','HH','tax','invest'])

*inspect the variables, sets, subsets etc. in the database:*

In [23]:
display('Variables:',db.variables_flat,'Sets:',db.sets_flat)

'Variables:'

['vS', 'vD', 'Peq', 'PwT', 'PbT', 'qD', 'qS']

'Sets:'

['alias_',
 'alias_set',
 'alias_map2',
 's',
 'n',
 's_prod',
 'n_prod',
 'n_fg',
 's_for',
 'n_for',
 'sfor_ndom',
 'sfor_nfor',
 's_HH',
 'inp_HH',
 'out_HH',
 'n_tax',
 's_tax',
 's_itory',
 's_inv',
 'inv',
 'dur2inv',
 'dur',
 'itoryD',
 'n_equi',
 'd_Peq',
 'd_vS',
 'd_vD',
 'd_PwT',
 'd_PbT',
 'd_qD',
 'd_qS',
 'd_tauS',
 'd_tauD',
 'd_tauLump']

*Example the variable qD (quantity, demand):*

In [24]:
db.get('qD')

s      n   
F      a          6.10701
       b           2.9872
HH     a          2.50754
       a_F        1.01495
       b          9.90321
       b_F      0.0313868
I_B    a          1.59205
       a_F      0.0124715
       b         0.126531
       b_F     0.00233702
I_M    a         0.819767
       a_F       0.605856
       b         0.652799
       b_F       0.153759
a      I_iB      0.998536
       I_iM       1.21727
       L          3.80338
       a           3.5306
       a_F        2.51619
       b          2.48622
       b_F        0.48449
       iB           2.092
       iM           1.046
b      I_iB      0.998536
       I_iM       1.21727
       L          7.02874
       a          1.37205
       a_F       0.603783
       b          20.9559
       b_F         3.1756
       iB         1.05102
       iM         2.10204
itory  a        0.0885507
       a_F      0.0613292
       b        0.0046528
       b_F     0.00112875
Name: qD, dtype: object

### 1: Condition trees

*The condition tree methods are generally build using nested dictionaries:* 
* *The keyword in the dictionary specifies the relevant operation used in the current nest ('and','or','not').*
* *The value specifies the boolean operation to perform. If more than one set of boolean conditions are given they should be defined in a list. (these can be specified in a number of ways)*
* *The boolean operations are specified based on the domains of the relevant symbols. As the examples show, this entails that we do not pass a vector of true/false; rather, we define the condition as a gpy\_symbol that is defined over a certain domain. The domains of the variable in question are then sliced according to the intersection of these domains.*
* *The condition trees can be nested by letting a value in one dictionary, be defined as a new dictionary.*


#### *1.1: Type of arguments*

*The condition tree can be build with three keywords ('and','or','not'). The values, specified in a list if more than one is applied, are generally defined as gpy\_symbols. The interpretation from a gpy\_symbol to a boolean condition is, however, quite flexible. Some examples:*

*1: Simple slicing using an index: The set 'sfor\_ndom' is a set defined over the same index (s,n) as the variable 'qD'.*

In [33]:
c = db['sfor_ndom']
display('The set:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

'The set:'

MultiIndex([('F', 'a'),
            ('F', 'b')],
           names=['s', 'n'])

'The sliced variable:'

s  n
F  a    6.10701
   b     2.9872
Name: qD, dtype: object

*2: Slicing using a partially overlapping index: The set 'n\_prod' is only defined over the set 'n'. The condition tree method looks for the overlap in domains and slice on this condition.*

In [41]:
c = db['n_prod']
display('The set:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

'The set:'

Index(['L', 'a', 'b', 'iB', 'iM'], dtype='object', name='n')

'The sliced variable:'

s      n 
F      a       6.10701
       b        2.9872
HH     a       2.50754
       b       9.90321
I_B    a       1.59205
       b      0.126531
I_M    a      0.819767
       b      0.652799
a      L       3.80338
       a        3.5306
       b       2.48622
       iB        2.092
       iM        1.046
b      L       7.02874
       a       1.37205
       b       20.9559
       iB      1.05102
       iM      2.10204
itory  a     0.0885507
       b     0.0046528
Name: qD, dtype: object

*3: Slicing using another variable: Similarly, a variable can be used to slice - this is done using the domains of the relevant variable.*

In [44]:
c = db['qS']
display('The slicing variable:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

'The slicing variable:'

s    n   
HH   L       10.8321
I_B  I_iB    1.99707
I_M  I_iM    2.43455
a    a       16.0176
b    b       37.1165
Name: qS, dtype: object

'The sliced variable:'

s  n
a  a     3.5306
b  b    20.9559
Name: qD, dtype: object

*4: The same conditions applied above can be written to a gams code 'condition' using the rctree\_gams method instead (should use indices instead of variables in the slicing though)*

In [50]:
c = db['n_prod']
db['qD'].rctree_gams(c)

'n_prod[n]'

#### *1.2: Negate conditions*

*The simplest condition tree is to negate a condition. This is simply done by adding a dictionary with the keyword 'not':*

In [55]:
c = {'not': db['n_prod']}
db['qD'].rctree_pd(c)

s      n   
HH     a_F        1.01495
       b_F      0.0313868
I_B    a_F      0.0124715
       b_F     0.00233702
I_M    a_F       0.605856
       b_F       0.153759
a      I_iB      0.998536
       I_iM       1.21727
       a_F        2.51619
       b_F        0.48449
b      I_iB      0.998536
       I_iM       1.21727
       a_F       0.603783
       b_F         3.1756
itory  a_F      0.0613292
       b_F     0.00112875
Name: qD, dtype: object

*We can also just add the value to a list:*

In [56]:
c = {'not': [db['n_prod']]}
db['qD'].rctree_pd(c)

s      n   
HH     a_F        1.01495
       b_F      0.0313868
I_B    a_F      0.0124715
       b_F     0.00233702
I_M    a_F       0.605856
       b_F       0.153759
a      I_iB      0.998536
       I_iM       1.21727
       a_F        2.51619
       b_F        0.48449
b      I_iB      0.998536
       I_iM       1.21727
       a_F       0.603783
       b_F         3.1756
itory  a_F      0.0613292
       b_F     0.00112875
Name: qD, dtype: object

*It is however not unambiguous how to interpret a list of more than one condition with the keyword not (should it be the union or intersection of 'True' to subset on?). With the keyword 'not', only the first argument of the list is used. Thus, this condition is the same as the one above:*

In [57]:
c = {'not': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

s      n   
HH     a_F        1.01495
       b_F      0.0313868
I_B    a_F      0.0124715
       b_F     0.00233702
I_M    a_F       0.605856
       b_F       0.153759
a      I_iB      0.998536
       I_iM       1.21727
       a_F        2.51619
       b_F        0.48449
b      I_iB      0.998536
       I_iM       1.21727
       a_F       0.603783
       b_F         3.1756
itory  a_F      0.0613292
       b_F     0.00112875
Name: qD, dtype: object

#### *1.3: List of and/or conditions*

*Assume instead that we want to subset on either one subset or the other. This is done with the keyword argument 'or':*

In [59]:
c = {'or': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

s      n  
F      a         6.10701
       b          2.9872
HH     a         2.50754
       a_F       1.01495
       b         9.90321
       b_F     0.0313868
I_B    a         1.59205
       a_F     0.0124715
       b        0.126531
       b_F    0.00233702
I_M    a        0.819767
       a_F      0.605856
       b        0.652799
       b_F      0.153759
a      L         3.80338
       a          3.5306
       a_F       2.51619
       b         2.48622
       b_F       0.48449
       iB          2.092
       iM          1.046
b      L         7.02874
       a         1.37205
       a_F      0.603783
       b         20.9559
       b_F        3.1756
       iB        1.05102
       iM        2.10204
itory  a       0.0885507
       a_F     0.0613292
       b       0.0046528
       b_F    0.00112875
Name: qD, dtype: object

*Using the keyword 'and' this returns an empty pandas series, as the two subsets ('n\_prod', 'n\_for') are not overlapping:*

In [62]:
c = {'and': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

Series([], Name: qD, dtype: object)

*As was the case earlier, this type of conditions can be written as gams code as well:*

In [63]:
db['qD'].rctree_gams(c)

'(n_prod[n] and n_for[n])'

#### *1.4: Nested conditions*

*Maybe we want the goods that are in one subset, but not the other. In this case, we make an 'and' condition, and nest the 'not' condition for the one subset. The condition tree traverses the nested dictionary to the most inner condition that are no longer nested, evaluate this to return a boolean condition, and loops out of the tree in this fashion. The following lookgs for goods that are in one set 'd\_tauD', but not in another 'd\_tauS':*

In [67]:
c = {'and': [db['d_tauD'], {'not': db['d_tauS']}]}
db['qD'].rctree_pd(c)

s      n   
F      a          6.10701
       b           2.9872
HH     a          2.50754
       a_F        1.01495
       b          9.90321
       b_F      0.0313868
I_B    a          1.59205
       a_F      0.0124715
       b         0.126531
       b_F     0.00233702
I_M    a         0.819767
       a_F       0.605856
       b         0.652799
       b_F       0.153759
a      I_iB      0.998536
       I_iM       1.21727
       L          3.80338
       a_F        2.51619
       b          2.48622
       b_F        0.48449
b      I_iB      0.998536
       I_iM       1.21727
       L          7.02874
       a          1.37205
       a_F       0.603783
       b_F         3.1756
itory  a        0.0885507
       a_F      0.0613292
       b        0.0046528
       b_F     0.00112875
Name: qD, dtype: object

*Just as before, we can write the corresponding gams condition using the rctree\_gams method:*

In [69]:
db['qD'].rctree_gams(c)

'(d_tauD[s,n] and not d_tauS[s,n])'